In [7]:
from fair_util import BMInterface
from fair_grid import BMGridSearch
from fair_bm import BMType
import pandas as pd

import mkl
mkl.set_num_threads(5)

train_df = pd.read_csv('./data/stroke_pre/stroke_train.csv')
val_df = pd.read_csv('./data/stroke_pre/stroke_val.csv')
test_df = pd.read_csv('./data/stroke_pre/stroke_test.csv')

# Defining the name of the label column
label_name = 'stroke'
# Defininf the name of the privileged group
sensitive_attribute = ['age']

bmI = BMInterface(train_df, val_df, test_df, label_name, sensitive_attribute)

privileged_group = [{'age': 1}]
unprivileged_group = [{'age': 0}]

In [4]:
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC  
from sklearn.linear_model import LogisticRegression
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier

mlp= MLPClassifier(
    max_iter=500,
    hidden_layer_sizes = (200,200,200,200),
    activation = 'logistic',
    solver = 'adam',
    alpha = 0.01,
    learning_rate = 'adaptive'
)

# Create a new instance of the classifier
lr = LogisticRegression(solver='liblinear')

xgbr =  XGBClassifier(
    max_depth=8,
    learning_rate=0.01,
    gamma = 0.25,
    n_estimators = 500,
    subsample = 0.8,
    colsample_bytree = 0.3)

tabnet = TabNetClassifier()

cat = CatBoostClassifier(eval_metric='Accuracy',
                         depth =  4,
                         learning_rate = 0.01,
                         iterations = 10)


model_list = [tabnet, xgbr, cat, mlp, lr]

C:\Users\enemy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
tabnet.__str__()#.startswith('XGBClassifier')

In [9]:
from itertools import product
from more_itertools import powerset

pre_bm = [BMType.preReweighing,BMType.preDisparate]
pos_bm = [BMType.posCalibrated, BMType.posEqqOds, BMType.posROC]

bm_list = [[pre, pos] for pre, pos in product(pre_bm, pos_bm)]
bm_list.extend([p] for p in pre_bm); bm_list.extend([p] for p in pos_bm)
bm_list

[[<BMType.preReweighing: 1>, <BMType.posCalibrated: 6>],
 [<BMType.preReweighing: 1>, <BMType.posEqqOds: 7>],
 [<BMType.preReweighing: 1>, <BMType.posROC: 8>],
 [<BMType.preDisparate: 2>, <BMType.posCalibrated: 6>],
 [<BMType.preDisparate: 2>, <BMType.posEqqOds: 7>],
 [<BMType.preDisparate: 2>, <BMType.posROC: 8>],
 [<BMType.preReweighing: 1>],
 [<BMType.preDisparate: 2>],
 [<BMType.posCalibrated: 6>],
 [<BMType.posEqqOds: 7>],
 [<BMType.posROC: 8>]]

In [17]:
import copy

bm_list_in = copy.deepcopy(bm_list)
[l.append(BMType.inAdversarial) for l in bm_list_in]
bm_list_in.extend(bm_list)
bm_list_in.append([BMType.inAdversarial])
bm_list_in

[[<BMType.preReweighing: 1>,
  <BMType.posCalibrated: 6>,
  <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.posEqqOds: 7>, <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.posROC: 8>, <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>,
  <BMType.posCalibrated: 6>,
  <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>, <BMType.posEqqOds: 7>, <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>, <BMType.posROC: 8>, <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>, <BMType.inAdversarial: 4>],
 [<BMType.posCalibrated: 6>, <BMType.inAdversarial: 4>],
 [<BMType.posEqqOds: 7>, <BMType.inAdversarial: 4>],
 [<BMType.posROC: 8>, <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.posCalibrated: 6>],
 [<BMType.preReweighing: 1>, <BMType.posEqqOds: 7>],
 [<BMType.preReweighing: 1>, <BMType.posROC: 8>],
 [<BMType.preDisparate: 2>, <BMType.posCalibrated: 6>],
 [<BMType.preDisparate: 

In [ ]:
for model in model_list:
    bmG = BMGridSearch(bmI, model=model, bm_list=bm_list, privileged_group=privileged_group, unprivileged_group=unprivileged_group)
    bmG.run_single_sensitive()

In [13]:
bmG = BMGridSearch(bmI, model=None, bm_list=[[BMType.inAdversarial]], privileged_group=privileged_group, unprivileged_group=unprivileged_group)
bmG.run_single_sensitive()

epoch 0; iter: 0; batch classifier loss: 0.745372
epoch 1; iter: 0; batch classifier loss: 0.612154
epoch 2; iter: 0; batch classifier loss: 0.635546
epoch 3; iter: 0; batch classifier loss: 0.565819
epoch 4; iter: 0; batch classifier loss: 0.600401
epoch 5; iter: 0; batch classifier loss: 0.577236
epoch 6; iter: 0; batch classifier loss: 0.567886
epoch 7; iter: 0; batch classifier loss: 0.506691
epoch 8; iter: 0; batch classifier loss: 0.465262
epoch 9; iter: 0; batch classifier loss: 0.546089
epoch 10; iter: 0; batch classifier loss: 0.468253
epoch 11; iter: 0; batch classifier loss: 0.522293
epoch 12; iter: 0; batch classifier loss: 0.659548
epoch 13; iter: 0; batch classifier loss: 0.436789
epoch 14; iter: 0; batch classifier loss: 0.508616
epoch 15; iter: 0; batch classifier loss: 0.490434
epoch 16; iter: 0; batch classifier loss: 0.357344
epoch 17; iter: 0; batch classifier loss: 0.367088
epoch 18; iter: 0; batch classifier loss: 0.472829
epoch 19; iter: 0; batch classifier loss:

In [19]:
for bm in bm_list:
    bmG = BMGridSearch(bmI, model=None, bm_list=bm_list_in, privileged_group=privileged_group, unprivileged_group=unprivileged_group)
    bmG.run_single_sensitive()

epoch 0; iter: 0; batch classifier loss: 0.719803
epoch 1; iter: 0; batch classifier loss: 0.654992
epoch 2; iter: 0; batch classifier loss: 0.610857
epoch 3; iter: 0; batch classifier loss: 0.581193
epoch 4; iter: 0; batch classifier loss: 0.558953
epoch 5; iter: 0; batch classifier loss: 0.539920
epoch 6; iter: 0; batch classifier loss: 0.502686
epoch 7; iter: 0; batch classifier loss: 0.493570
epoch 8; iter: 0; batch classifier loss: 0.614913
epoch 9; iter: 0; batch classifier loss: 0.539116
epoch 10; iter: 0; batch classifier loss: 0.472148
epoch 11; iter: 0; batch classifier loss: 0.496182
epoch 12; iter: 0; batch classifier loss: 0.460344
epoch 13; iter: 0; batch classifier loss: 0.436358
epoch 14; iter: 0; batch classifier loss: 0.418672
epoch 15; iter: 0; batch classifier loss: 0.394867
epoch 16; iter: 0; batch classifier loss: 0.549934
epoch 17; iter: 0; batch classifier loss: 0.408042
epoch 18; iter: 0; batch classifier loss: 0.423845
epoch 19; iter: 0; batch classifier loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


epoch 0; iter: 0; batch classifier loss: 0.679534; batch adversarial loss: 0.728273
epoch 1; iter: 0; batch classifier loss: 0.659544; batch adversarial loss: 0.700448
epoch 2; iter: 0; batch classifier loss: 0.649007; batch adversarial loss: 0.703333
epoch 3; iter: 0; batch classifier loss: 0.648143; batch adversarial loss: 0.708444
epoch 4; iter: 0; batch classifier loss: 0.623381; batch adversarial loss: 0.716429
epoch 5; iter: 0; batch classifier loss: 0.626076; batch adversarial loss: 0.708169
epoch 6; iter: 0; batch classifier loss: 0.587354; batch adversarial loss: 0.720147
epoch 7; iter: 0; batch classifier loss: 0.547467; batch adversarial loss: 0.715715
epoch 8; iter: 0; batch classifier loss: 0.607473; batch adversarial loss: 0.717915
epoch 9; iter: 0; batch classifier loss: 0.617357; batch adversarial loss: 0.696319
epoch 10; iter: 0; batch classifier loss: 0.581906; batch adversarial loss: 0.699845
epoch 11; iter: 0; batch classifier loss: 0.611414; batch adversarial loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


epoch 0; iter: 0; batch classifier loss: 0.713886; batch adversarial loss: 0.796376
epoch 1; iter: 0; batch classifier loss: 0.697121; batch adversarial loss: 0.921602
epoch 2; iter: 0; batch classifier loss: 0.727936; batch adversarial loss: 0.923898
epoch 3; iter: 0; batch classifier loss: 0.700186; batch adversarial loss: 1.038342
epoch 4; iter: 0; batch classifier loss: 0.704873; batch adversarial loss: 1.086937
epoch 5; iter: 0; batch classifier loss: 0.812813; batch adversarial loss: 1.227965
epoch 6; iter: 0; batch classifier loss: 0.864645; batch adversarial loss: 1.397516
epoch 7; iter: 0; batch classifier loss: 0.767736; batch adversarial loss: 1.227604
epoch 8; iter: 0; batch classifier loss: 0.852062; batch adversarial loss: 1.208915
epoch 9; iter: 0; batch classifier loss: 0.796743; batch adversarial loss: 1.240277
epoch 10; iter: 0; batch classifier loss: 1.093536; batch adversarial loss: 1.396934
epoch 11; iter: 0; batch classifier loss: 0.918514; batch adversarial loss:

...

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


epoch 0; iter: 0; batch classifier loss: 0.650862; batch adversarial loss: 0.739318
epoch 1; iter: 0; batch classifier loss: 0.659691; batch adversarial loss: 0.789088
epoch 2; iter: 0; batch classifier loss: 0.675921; batch adversarial loss: 0.799783
epoch 3; iter: 0; batch classifier loss: 0.646468; batch adversarial loss: 0.793489
epoch 4; iter: 0; batch classifier loss: 0.686371; batch adversarial loss: 0.820268
epoch 5; iter: 0; batch classifier loss: 0.681104; batch adversarial loss: 0.845017
epoch 6; iter: 0; batch classifier loss: 0.847352; batch adversarial loss: 0.931536
epoch 7; iter: 0; batch classifier loss: 0.856703; batch adversarial loss: 0.894274
epoch 8; iter: 0; batch classifier loss: 0.752944; batch adversarial loss: 0.865300
epoch 9; iter: 0; batch classifier loss: 0.743242; batch adversarial loss: 0.845164
epoch 10; iter: 0; batch classifier loss: 0.851550; batch adversarial loss: 0.896326
epoch 11; iter: 0; batch classifier loss: 0.968200; batch adversarial loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


epoch 0; iter: 0; batch classifier loss: 0.719228; batch adversarial loss: 0.740388
epoch 1; iter: 0; batch classifier loss: 0.708387; batch adversarial loss: 0.739791
epoch 2; iter: 0; batch classifier loss: 0.648779; batch adversarial loss: 0.748028
epoch 3; iter: 0; batch classifier loss: 0.658029; batch adversarial loss: 0.687131
epoch 4; iter: 0; batch classifier loss: 0.657254; batch adversarial loss: 0.733654
epoch 5; iter: 0; batch classifier loss: 0.590790; batch adversarial loss: 0.748695
epoch 6; iter: 0; batch classifier loss: 0.593804; batch adversarial loss: 0.769261
epoch 7; iter: 0; batch classifier loss: 0.658408; batch adversarial loss: 0.762331
epoch 8; iter: 0; batch classifier loss: 0.508819; batch adversarial loss: 0.865844
epoch 9; iter: 0; batch classifier loss: 0.545554; batch adversarial loss: 0.767737
epoch 10; iter: 0; batch classifier loss: 0.546303; batch adversarial loss: 0.821102
epoch 11; iter: 0; batch classifier loss: 0.569473; batch adversarial loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)
